# Εργαστήριο 10
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Άσκηση 1: αποκωδικοποίηση εντολών
Στην απλή ΚΜΕ που σχεδιάζετε, το κύκλωμα της **αποκωδικοποίησης** των εντολών μηχανής θα μετατρέπει τα bits των υποστηριζόμενων εντολών στα αντίστοιχα σήματα ελέγχου προς τα κυκλώματα εκτέλεσης των πράξεων και το register file του datapath.

Στο σημείο αυτό θα πρέπει να προσδιοριστούν ακριβώς (κι όχι παραμετρικά όπως σε προηγούμενα εργαστήρια) τα χαρακτηριστικά της ΚΜΕ υπό σχεδίαση. Τα χαρακτηριστικά αυτά θα είναι:

* 8 καταχωρητές γενικού σκοπού, των 16 bits ο καθένας.
* Program counter επίσης των 16 bits.
* Εντολές σταθερού μήκους, 16 bits η καθεμία.

### Κωδικοποίηση εντολών
Έστω ότι το σύστημά σας διαθέτει εντολές σταθερού μήκους **16 bits** και αναγνωρίζει (προς το παρόν) 2 κατηγορίες εντολών:
* Αριθμητικές-λογικές πράξεις
* Ανάθεση σταθεράς σε καταχωρητή

#### Αριθμητικές-λογικές πράξεις
Οι αριθμητικές-λογικές πράξεις έχουν τη μορφή:

`Rdst <- Rsrc1 Funct Rsrc2`

όπου είναι:

* `Rdst`: καταχωρητής προορισμού αποτελέσματος (3 bits)
* `Rsrc1`: καταχωρητής εισόδου δεδομένων 1 (3 bits)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (3 bits)
* `Funct`: bits επιλογής πράξης (4 bits)

Η κωδικοποίηση των εντολών αυτών έχει ως εξής:

| `bits 15-13` | `bits 12-9` | `bits 8-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- | --- |
| `000` | `Funct` | `Rsrc1` | `Rsrc2` | `Rdst` |


Τα bits επιλογής πράξης (`Funct`) έχουν την ακόλουθη μορφή:

| `Funct` | `Πράξη` |
| - | - |
| `0000` | `AND` |
| `0001` | `OR` |
| `0010` | `XOR` |
| `0011` | `+` |
| `0111` | `-` |

Παρατηρήστε ότι τα bits 0-1 του `Funct` (bits 9-10 της συνολικής εντολής) αντιστοιχούν στο σήμα `sel` του datapath ενώ το bit 2 (bit 11 της συνολικής εντολής) αντιστοιχεί στο σήμα `sub` του datapath.

### Ανάθεση σταθεράς (immediate) σε καταχωρητή
H μορφή αυτής της εντολής είναι η ακόλουθη:

`Rdst <- Immediate`

όπου:

* `Rdst`: καταχωρητής προορισμού αποτελέσματος (3 bits)
* `Immediate`: σταθερά 10 bits

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-13` | `bits 12-3` | `bits 2-0` |
| --- | --- | --- |
| `001` | `Immediate` | `Rdst` |

**Προσοχή:** Το μήκος της σταθεράς immediate είναι 10 bits, οπότε πριν την αποθήκευση στον `Rdst` θα πρέπει να επεκταθεί στα 16 bits (εύρος καταχωρητών). Η επέκταση γίνεται λαμβάνοντας υπόψη το πρόσημο (bit 9) της σταθεράς για να μπορούμε να χρησιμοποιήσουμε και αρνητικούς αριθμούς (*ο κώδικας για την επέκταση προσήμου δίνεται*).


### Ζητούμενο
Το κύκλωμα αποκωδικοποίησης (module `DecodeUnit`) θα δέχεται ως είσοδο τα bits κάθε εντολής (είσοδος `instruction`, εύρος 16 bits) και θα παράγει τα σήματα ελέγχου του module `DataPath`, δηλαδή τα εξής:
* `im`, εύρος 16 bits
* `sel`, εύρος 2 bits
* `sub`, εύρος 1 bit
* `alu_a_sel`, εύρος 1 bit
* `read_sel_a`, εύρος 3 bits
* `read_sel_b`, εύρος 3 bits
* `write_sel`, εύρος 3 bits

Στο επόμενο κελί κάποια από τα ζητούμενα έχουν ήδη δοθεί ως υπόδειγμα. Συμπληρώστε τα σημεία που λείπουν: 

In [ ]:
class DecodeUnit extends Module {
  val io = IO(new Bundle {
    // input  
    val instruction = Input(UInt(16.W))
    // outputs  
    val im = Output(UInt(16.W))
    val sel = Output(UInt(2.W))
    val sub = Output(UInt(1.W))
    var alu_a_sel = Output(UInt(1.W))
    val read_sel_a = Output(UInt(3.W))
    val read_sel_b = Output(UInt(3.W))
    val write_sel = Output(UInt(3.W))
  })
    
  // output default values
  io.write_sel := 0.U
  io.read_sel_a := io.instruction(8,6)
  io.read_sel_b := io.instruction(5,3)
  io.alu_a_sel := 1.U
  io.im := 0.U
  io.sel := 0.U
  io.sub := 0.U
    
  // decoding of instructions
  val instr_type = io.instruction(15,13)
  
  switch(instr_type) {
    is("b000".U) {  // logic-arithmetic
      io.sel := io.instruction(10,9)
      io.sub := // ..συμπληρώστε..
      io.write_sel := // ..συμπληρώστε..  
    }
    is("b001".U) {  // load immediate
      io.im := io.instruction(12,3).asSInt.pad(16).asUInt  // sign-extend to 16 bits
      
      // ..συμπληρώστε κατάλληλα δίνοντας τιμές στα σήματα io.alu_a_sel, io.read_sel_b, io.sel και io.write_sel..
        
    }  
  }  
}

In [ ]:
test(new DecodeUnit) { c =>
  // test logic-arithmetic instr
  c.io.instruction.poke("b0000011001010011".U)  // r3 = r1 + r2
  c.io.alu_a_sel.expect(1.U)
  c.io.read_sel_a.expect(1.U)
  c.io.read_sel_b.expect(2.U)
  c.io.sel.expect("b11".U)
  c.io.sub.expect(0.U)  
  c.io.write_sel.expect(3.U)    
  // test load immediate instr  
  c.io.instruction.poke("b0010000100001001".U)  // r1 = 33
  c.io.im.expect("b0000000000100001".U)
  c.io.alu_a_sel.expect(0.U)
  c.io.read_sel_b.expect(0.U)
  c.io.sel.expect("b01".U)
  c.io.write_sel.expect(1.U)
  // test sign extension in load immediate instr  
  c.io.instruction.poke("b0011111011110010".U)  // r2 = -34
  c.io.im.expect("b1111111111011110".U)
}
println("SUCCESS!!")

## Προαπαιτούμενα για την επόμενη άσκηση
Για τη συνέχεια θα χρειαστείτε:
* Το module `DataPath` ([εργαστήριο 8](https://nbviewer.org/urls/mixstef.github.io/courses/comparch/lab8.ipynb)) και τα submodules αυτού, `Alu` και `Registerfile`.
* Το module `FetchUnit` ([εργαστήριο 9](https://nbviewer.org/urls/mixstef.github.io/courses/comparch/lab9.ipynb)) και το submodule αυτού, `InstructionMemory`.

Αντιγράψτε το κώδικά σας για τα modules αυτά στα επόμενα κελιά:

In [ ]:
class RegisterFile(register_width: Int, register_number: Int) extends Module {
  
  // ..συμπληρώστε (από προηγούμενο εργαστήριο)..
    
}

In [ ]:
class Alu(n: Int) extends Module {
    
  // ..συμπληρώστε (από προηγούμενο εργαστήριο)..
}

In [ ]:
class DataPath(word_width: Int, register_number: Int) extends Module {
    
  // ..συμπληρώστε (από προηγούμενο εργαστήριο)..
    
}

In [ ]:
class InstructionMemory(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {
    
  // ..συμπληρώστε (από προηγούμενο εργαστήριο)..
    
}

In [ ]:
class FetchUnit(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {
    
  // ..συμπληρώστε (από προηγούμενο εργαστήριο)..
    
}

## Άσκηση 2: η πρώτη μορφή της ΚΜΕ
Ζητούμενο είναι ο συνδυασμός των modules `FetchUnit`, `DecodeUnit` και `Datapath` για την κατασκευή της 1ης μορφής μιας απλής ΚΜΕ (θα επεκταθεί σε επόμενα εργαστήρια) σε νέο module με ονομασία `Cpu`.

Στο νέο module η μόνη παράμετρος εισόδου είναι η ακολουθία `instructions` που θα παρέχει τα περιεχόμενα της μνήμης εντολών. Τα υπόλοιπα χαρακτηριστικά της ΚΜΕ είναι σταθερά ορισμένα (λόγω του module `DecodeUnit`):

* Το εύρος λέξης είναι 16 bits, τό ίδιο και το εύρος του program counter.
* Υπάρχουν 8 καταχωρητές γενικού σκοπού.
* Κάθε εντολή είναι 16 bits.

Συμπληρώστε τα σημεία που λείπουν στο επόμενο κελί, συνδέοντας μεταξύ τους τα modules `FetchUnit`, `DecodeUnit` και `Datapath` (τα σήματα δηλαδή με το ίδιο όνομα):

In [ ]:
class Cpu(instructions: Seq[UInt]) extends Module {
  val io = IO(new Bundle {
    // no inputs/outputs
      
    // output for debugging
    val results = Output(UInt(16.W))
      
  })
  
  val fUnit = Module(new FetchUnit(16,16,instructions))
  val dUnit = Module(new DecodeUnit)
  val dPath = Module(new DataPath(16,8))
  
  // these signals will be defined later
  fUnit.io.pc_sel := 0.U
  fUnit.io.branch_pc := 0.U
    
  // connect fetched instruction to decoder
    
  // ..συνδέστε το σήμα instruction του fetch unit με το ομώνυμο του decode unit..   
  
    
  // connect decoded output signals to datapath input counterparts
    
  // ..συνδέστε τα σήματα im, alu_a_sel, read_sel_a, read_sel_b, write_sel, sel και sub
  // του decode unit με τα ομώνυμα του datapath..   
    
  // connect debugging output
  io.results := dPath.io.results  
}

### Έλεγχος
Το παρακάτω κελί περιέχει τον κώδικα για την εκτέλεση 3 εντολών στην απλή ΚΜΕ που σχεδιάσατε. Παρατηρήστε ότι η 1η εντολή εκτελείται στην αρχή, *πριν το πρώτο βήμα ρολογιού*. Έτσι χρειάζονται μόνο 2 βήματα ρολογιού (`c.clock.step()`) για να ολοκληρωθούν και οι 3 εντολές.

Εάν τα προηγούμενα modules που φτιάξατε είναι λειτουργικά ορθά, ο επόμενος έλεγχος θα πρέπει να είναι επιτυχής:

In [ ]:
val instructions = List("b0010000100001001".U,  // r1 = 33
                        "b0011111011110010".U,  // r2 = -34
                        "b0000011001010011".U,  // r3 = r1 + r2
                       )
test(new Cpu(instructions)) { c =>
  for (i <- 0 until instructions.length-1) { // nth instruction executed after clock n-1
    c.clock.step()
  }
  c.io.results.expect("b1111111111111111".U) // 33 + (-34) = -1  
}
println("SUCCESS!!")

## Ανεβάστε τη λύση σας στο opencourses
Ανεβάστε το τελικό σας notebook στο opencourses (**Εργασία 3**) έως και τη **Δευτέρα 19/12**.

*(Ίσως χρειαστεί να κάνετε zip το αρχείο ipynb για να γίνει αποδεκτό από το σύστημα)*
